## Matching using Z3

Consider a set of employees and a set of tasks. Suppose that each employee chooses which task they are willing to do. We can model this relationship using the following graph, where left vertices represent employees and right vertices represent tasks. Edges represent employee preferences: an employee and task are joined by an edge if the employee is okay with doing the task.


In [ ]:
employees = [ 'Alice', 'Bob', 'Carl' ]
tasks = [ 'Print', 'Talk', 'Write' ]

taskPreferences = [
    ( 'Alice', 'Print' ), # Alice can perform the Print task
    ( 'Alice', 'Talk' ),
    ( 'Alice', 'Write' ),
    ( 'Bob', 'Talk' ),
    ( 'Carl', 'Print' ),
    ( 'Carl', 'Talk' )
]
draw_bipartite_graph( employees, tasks, taskPreferences )

We want to assign tasks to each employee so that each task is assigned to an employee, and each employee has at most one task assigned to them, i.e., we want to **match** employees and tasks such that each task is assigned to an employee.

Formally, a **matching** is a collection of edges where no two vertices share an edge. We want to find a matching where each vertex on the right is part of an edge in the matching.

Let's see how to do that using Z3.

<!-- For the sake of completeness, we now provide some formal definitions. A bipartite graph  -->
<!-- Formally, given two disjoint sets of vertices $V_1$ and $V_2$ in a graph, a **matching** is a set of edges $M$ such that: (1) no two edges in $M$ share a vertex, and (2) each edge in $M$ has one vertex in $V_1$ and one vertex in $V_2$. -->

#### Step 1: Make boolean variables for each edge

In our solution, an edge with the value of True will represent an edge that has been selected in our matching.

In [ ]:
# Alice shares an edge with Print, we will represent that edge with the variable AlicePrint:
AlicePrint = Bool( 'AlicePrint' )

# We follow the same naming convention for our other variables

AliceTalk = Bool( 'AliceTalk' )
AliceWrite = Bool( 'AliceWrite' )
BobTalk = Bool( 'BobTalk' )
CarlPrint = Bool( 'CarlPrint' )
CarlTalk = Bool( 'CarlTalk' )

#### Step 2: Initialize solver

In [ ]:
s = Solver()

#### Step 3: Write constraints

In [ ]:
# Each left vertex (Employee) can have at most one edge adjacent to it in the matching
s.add( AlicePrint + AliceTalk + AliceWrite <= 1 )
s.add( CarlPrint + CarlTalk <= 1 )
# Note that we do not have to add the constraint "Bob_Talk <= 1" since Bob_Talk
# is a boolean variable and its value is always at most 1 when converted to
# an integer

# Each right vertex (Task) can have at most one edge adjacent to it in the matching
s.add( AlicePrint + CarlPrint <= 1 )
s.add( AliceTalk + BobTalk + CarlTalk <= 1 )

# Now we need to add a constraint that ensures all of the right vertices are
# part of an edge in the matching. Since there are three tasks, we can do this
# by asking for the sum of all edges going into these tasks be at least three:

s.add( AlicePrint + CarlPrint + AliceTalk + BobTalk + CarlTalk + AliceWrite >= 3 )

# Let's see what our constraints look like
showSolver( s )

#### Step 4: Check if a solution exists

In [ ]:
print( s.check() )

####Step 5: View solution

In [ ]:
solution = s.model()
print(solution)

We have also defined a function to visualize our solution:

In [ ]:
print_matching_solution( solution )